In [2]:
!pip install selenium

In [2]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import re

# 크롬창(웹드라이버) 열기
driver = webdriver.Chrome("./chromedriver")

# 구글 지도 접속하기
driver.get("https://www.google.com/maps/?hl=en")

# 검색하기
searchbox = driver.find_element(By.CSS_SELECTOR,"input#searchboxinput")
searchbox.send_keys("N Seoul Tower")

time.sleep(5)
check=input('일치하는 장소 선택 후 y를 눌러주세요(이외 멈춤)')
if check.upper()=='Y':
    now_url=driver.current_url

# 검색버튼 누르기
searchbutton = driver.find_element(By.CSS_SELECTOR,"button#searchbox-searchbutton")
searchbutton.click()
time.sleep(10)
reviewbutton = driver.find_element(By.CSS_SELECTOR,"button.DkEaL")
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
num_review=int(float(soup.find('button',{'class':'DkEaL'}).get_text()[:-8].replace(',', '')))
reviewbutton.click()
time.sleep(15)

#최신순 정렬
sortbutton=driver.find_element(By.XPATH,"//*[@id='QA0Szd']/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[7]/div[2]/button")
sortbutton.click()
time.sleep(3)
newestbutton=driver.find_element(By.XPATH,"//*[@id='action-menu']/ul/li[2]")
newestbutton.click()
time.sleep(3)
#스크롤
for i in range(91): #구글맵 크롤링 최대 930개
    action = ActionChains(driver)
    last = driver.find_element(By.XPATH,"//*[@id='QA0Szd']/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]/div[{}]".format(30*(i+1)-1))
    all=driver.find_element(By.XPATH,"//*[@id='QA0Szd']/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]")
    action.move_to_element(last).perform()
    all.send_keys(Keys.PAGE_DOWN)
    print("*",end='')
    try:
        WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,"//*[@id='QA0Szd']/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]/div[{}]".format(30*(i+2)-1))))
    except:
        break
# 페이지 소스 가져오기
reviews = driver.find_elements(By.CSS_SELECTOR,'.jftiEf')
name=[]
star=[]
content=[]
date=[]
for review in reviews:
    try:
        action.move_to_element(review).perform()
        morereviewbutton=review.find_element(By.CSS_SELECTOR,"button.w8nwRe.kyuRq")
        print("y",end="")
        morereviewbutton.click()
        WebDriverWait(review,60).until(EC.invisibility_of_element_located((By.CSS_SELECTOR,"button.w8nwRe.kyuRq")))
        name.append(review.find_element(By.CSS_SELECTOR,'.d4r55').text)
        star.append(int(float(review.find_element(By.CSS_SELECTOR,'.kvMYJc').get_attribute("aria-label")[1])))
        content.append(review.find_element(By.CSS_SELECTOR,'.wiI7pd').text.replace("\n"," "))
        date.append(review.find_element(By.CSS_SELECTOR,'.rsqaWe').text)
    except:
        print("n",end="")
        name.append(review.find_element(By.CSS_SELECTOR,'.d4r55').text)
        star.append(int(float(review.find_element(By.CSS_SELECTOR,'.kvMYJc').get_attribute("aria-label")[1])))
        content.append(review.find_element(By.CSS_SELECTOR,'.wiI7pd').text.replace("\n"," "))
        date.append(review.find_element(By.CSS_SELECTOR,'.rsqaWe').text)
all=pd.DataFrame([name,star,content,date],index=['이름','별점','리뷰','날짜']).transpose()
print(content)
emoji = re.compile("["
                    u"\U0001F600-\U0001F64F"
                    u"\U0001F300-\U0001F5FF"
                    u"\U0001F680-\U0001F6FF"
                    u"\U0001F1E0-\U0001F1FF"
                       "]+", flags=re.UNICODE)
        
#이모티콘,특수문자, 한글 제거
content_noemj=[]
name_noko=[]
korean = re.compile('[\u3131-\u3163\uac00-\ud7a3]+')
for i in name :
    i = re.sub(korean, ' KoReAn ', i)
    i = i.encode('utf-8').decode('ascii','ignore')
    name_noko.append(i)
            
for i in content :
    i = emoji.sub(r'',i)
    i = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\₩[\]\<\>`\'…《\》’]', '', i)
    i = re.sub(korean, ' KoReAn ', i) 
    i = i.encode('utf-8').decode('ascii','ignore')
    content_noemj.append(i)
print(len(content),len(name))
all=pd.DataFrame([name_noko,star,content_noemj,date],index=['NAME','STAR','CONTENT','DATE']).transpose()
print(content_noemj)
all.to_csv('Seoul Botanic Park review.csv')

C:\Users\wndgm\AppData\Local\Temp/ipykernel_36076/807747869.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./chromedriver")


일치하는 장소 선택 후 y를 눌러주세요(이외 멈춤)y
*********************************************************ynnnnynnnynynnynnynnnynynynnnnynnnnynynynynnyynynynnnynnnynynnnynnynnnynnnnnynnynnynynynnnnynnnnnnnynynynynynynynnynynynnnynyyynnynnyynyyynynyyyyyynynnnynnynynynnnynynynnynnynynnnnnnynynynynnynnynynynnynnynynnynynynynnynnnynnynynynynynynynnynnnynnnynnnynynnnynnynnynnynynynnnnnynynnynynnnnynynynynynynynynnynnynnynnynynnnnnynnynnnnynnynynnnynnnnynynynnynynnynynnynyyynnnyyynnynyyyynynyyyynnynyyynnnyynnnnynnnynyynyyyyyyyynnnnynnnnyynnynnynyynnynynyynnnynynyyynynyyyyyynyyyyynynyynyyyynynnnynyynnnyynnyynyynyyyynnnnynyyynnynnnnnnnynnnyynyyynnnyynyyyynynyynynyynyyyyyyynnynnnnnyyynnnnnnyyyyynyynnnyyynyyyyyynyyynnyynyynnnnnnyyynyynyynnyynynnynnyyynnnnnyyynnnnyyynnynynnnnnynynyyynnnnyyyyyyyyynnynny['', 'Namsan tower ❤️ nice view mostly in the night 🇰🇷✨', '', '', '(Translated by Google) The views are great. You should come by bus number 1 or the cable car track. Otherwise, it can get hot climbing the mountain. …

In [3]:
def review_csv(filename):
    for file in filename:
        df = pd.read_csv(file)
        df = df.drop(df.columns[[0,1]], axis=1)
        df = df[~df['CONTENT'].str.contains("KoReAn", na=False, case=False)]
        df['CONTENT'].replace('',np.nan,inplace=True)
        df=df.dropna(axis=0)
        content_re=[]
        for i in df['CONTENT']:
            try:
                m = re.search('Translated by Google (.+?)  Original ', str(i))
                if m:
                    m = m.group(1)
                    content_re.append(m)
                else: 
                    i = re.sub("Translated by Google","",i)
                    content_re.append(i)
            except :
                content_re.append(i)

        df['CONTENT'] = content_re
        df = df.reset_index()
        df = df.drop(df.columns[[0]], axis=1)
            
        csv_name = '{}{}'.format('final ',file)
        df.to_csv(csv_name)

In [4]:
filename = ['N Seoul Tower review.csv']
review_csv(filename)